# 第8章: ニューラルネット

[https://nlp100.github.io/ja/ch08.html](https://nlp100.github.io/ja/ch08.html)

第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

## 70. 単語ベクトルの和による特徴量

In [5]:
import gensim
import numpy as np
import pandas as pd
import spacy
import torch
import tqdm

# global
dataset_types = ['train', 'valid', 'test']

def makeDatasetFiles():
    nlp = spacy.load("en_core_web_sm")

    # Word2Vec
    w2v = gensim.models.KeyedVectors.load_word2vec_format(
        '../Chapter7/GoogleNews-vectors-negative300.bin', 
        binary=True)

    # Load texts and make tensors
    Xs, ys = {}, {}
    dataset_types = ['train', 'valid', 'test']
    label2int = {
        "b": 0,
        "t": 1,
        "e": 2,
        "m": 3
    }

    for dataset in dataset_types:
        tmp_x, tmp_y = [], []
        tmp_df = pd.read_table('../Chapter6/{:}.txt'.format(dataset))

        for each in tmp_df.itertuples():

            # make X
            tokens = [token for token in nlp(each.TITLE)]
            num_tokens = len(tokens)

            x_i = np.zeros(300)
            for token in tokens:
                try:
                    token_embedding = w2v[str(token)]
                    x_i = np.add(x_i, token_embedding)

                except KeyError:
                    num_tokens -= 1
                    continue

            x_i = np.divide(x_i, num_tokens)
            tmp_x.append(x_i)

            # make y
            tmp_y.append(label2int[each.CATEGORY])
        
        # convert to torch.Tensor
        Xs[dataset] = torch.Tensor(tmp_x)
        ys[dataset] = torch.Tensor(tmp_y)

        # pickle
        torch.save(Xs[dataset], 'X_{:}.pickle'.format(dataset))
        torch.save(ys[dataset], 'y_{:}.pickle'.format(dataset))
    
    return Xs, ys

## 71. 単層ニューラルネットワークによる予測

In [7]:
try:
    for dataset in dataset_types:
        Xs[dataset] = torch.load('X_{:}.pickle'.format(dataset))
        ys[dataset] = torch.load('y_{:}.pickle'.format(dataset))
except FileNotFoundError:
    Xs, ys = makeDatasetFiles()
    assert Xs != {} and ys != {}

